我们想对函数 $y=2\mathbf{x}^\top\mathbf{x}$ 关于列向量 $\mathbf{x}$ 求导

In [12]:
import torch
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

计算y关于x的梯度之前，我们需要一个地方来存储梯度。

In [13]:
x.requires_grad_(True) #等价于 x = torch.arange(4.0, requires_grad=True)
x.grad #默认是None

现在计算y

In [14]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

通过调用反向传播函数来自动计算y关于x每个分量的梯度

In [15]:
y.backward() #自动求导
x.grad

tensor([ 0.,  4.,  8., 12.])

In [16]:
x.grad == 4 * x

tensor([True, True, True, True])

现在计算x的另一个函数

In [18]:
# 在默认情况下，PyTorch会累积梯度，我们要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

深度学习中，我们的目的不是计算微分矩阵，而是批量中每个样本独立的偏导数之和

In [19]:
# 对非标量变量进行反向传播时，需要传入一个与该变量同形的梯度参数
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

可以将某些计算移动到记录的计算图之外


In [21]:
x.grad.zero_()
y = x * x
u = y.detach()  # 中断y对x的依赖关系
z = u * x
z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [22]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

即使构建函数的计算图需要通过Python控制流（例如、条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度

In [58]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()
a.grad == d / a,a

(tensor(True), tensor(-0.6260, requires_grad=True))